# Predicción de un Activo Financiero con Minería de Datos

En este notebook aprenderemos a:
1. Descargar y graficar datos de un activo financiero con `yfinance`.
2. Analizar los datos y calcular estadísticas básicas.
3. Preparar variables explicativas (features).
4. Dividir en datos de entrenamiento y prueba.
5. Aplicar modelos de predicción: Regresión Lineal, Random Forest y ARIMA.
6. Evaluar y comparar resultados.
7. Responder preguntas guía para interpretar el ejercicio.

---


In [ ]:
# Instalar librerías necesarias (ejecutar si falta alguna)
# !pip install yfinance pandas numpy matplotlib scikit-learn statsmodels


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings

warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (10,6)


## 1. Descargar y visualizar datos del activo

In [ ]:
TICKER = "AAPL"
YEARS = 5

try:
    data = yf.download(TICKER, period=f"{YEARS}y")
except Exception as e:
    print("No se pudieron descargar datos reales. Generando datos sintéticos...")
    dates = pd.date_range(start="2020-01-01", periods=1000, freq="B")
    np.random.seed(42)
    prices = np.cumsum(np.random.normal(0,1,len(dates))) + 100
    data = pd.DataFrame({"Close": prices}, index=dates)

data = data[['Close']].dropna()
data.head()


In [ ]:
data['Close'].plot(title=f"Precio de cierre de {TICKER}")
plt.show()


## 2. Análisis de datos y estadísticas básicas

In [ ]:
rets = data['Close'].pct_change().dropna()

total_return = data['Close'][-1]/data['Close'][0] - 1
annual_return = (1+total_return)**(252/len(data)) - 1
volatility = rets.std()*np.sqrt(252)
max_dd = (data['Close']/data['Close'].cummax() - 1).min()

print("Rendimiento total: {:.2%}".format(total_return))
print("Rendimiento anualizado: {:.2%}".format(annual_return))
print("Volatilidad anualizada: {:.2%}".format(volatility))
print("Máxima caída (drawdown): {:.2%}".format(max_dd))

rets.plot(title="Rendimientos diarios (%)")
plt.show()


## 3. Preparación de variables explicativas (features)

In [ ]:
df = data.copy()
df['Return'] = df['Close'].pct_change()
df['Lag1'] = df['Close'].shift(1)
df['Lag2'] = df['Close'].shift(2)
df['Volatility20'] = df['Return'].rolling(20).std()
df['Volume'] = np.random.randint(1e6,5e6,len(df))  # proxy simple
df['Target'] = df['Close'].shift(-1)
df = df.dropna()
df.head()


## 4. División en entrenamiento y prueba

In [ ]:
TEST_DAYS = 90
train, test = df.iloc[:-TEST_DAYS], df.iloc[-TEST_DAYS:]
X_train, y_train = train[['Lag1','Lag2','Volatility20','Volume']], train['Target']
X_test, y_test = test[['Lag1','Lag2','Volatility20','Volume']], test['Target']


## 5. Modelos de predicción

In [ ]:
models = {}

# Regresión Lineal
lr = LinearRegression().fit(X_train, y_train)
models['Linear Regression'] = lr

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
models['Random Forest'] = rf

# ARIMA
try:
    from statsmodels.tsa.arima.model import ARIMA
    arima_model = ARIMA(train['Close'], order=(1,1,1)).fit()
    arima_forecast = arima_model.forecast(steps=len(test))
    models['ARIMA'] = arima_forecast
except:
    print("Statsmodels no disponible, se omite ARIMA")


## 6. Evaluación de modelos

In [ ]:
for name, model in models.items():
    if name == 'ARIMA':
        preds = model.values
    else:
        preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    mae = mean_absolute_error(y_test, preds)
    print(f"{name} -> RMSE: {rmse:.2f}, MAE: {mae:.2f}")
    
    plt.plot(y_test.index, y_test, label="Real")
    plt.plot(y_test.index, preds, label="Predicción")
    plt.title(name)
    plt.legend()
    plt.show()


## 7. Preguntas guía para el análisis

In [ ]:
print("1. ¿Qué periodo estamos observando?")
print(f"   Desde {data.index[0].date()} hasta {data.index[-1].date()}")

print("\n2. ¿Por qué ese activo y horizonte de tiempo?")
print("   Porque es una acción representativa y un periodo suficientemente largo para observar ciclos.")

print("\n3. ¿Qué observamos en la tendencia y volatilidad?")
print("   Los precios muestran fluctuaciones, con periodos de crecimiento y caídas; la volatilidad refleja los riesgos.")

print("\n4. ¿Qué limitaciones tienen los modelos?")
print("   La linealidad en la regresión, el sobreajuste en Random Forest y la suposición de estacionariedad en ARIMA.")

print("\n5. ¿Cómo mejorar el pronóstico?")
print("   Usando más datos, variables externas (macro, sentimiento), optimización de hiperparámetros o modelos más complejos como LSTM.")
